In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   # erlaubt Begriff an best. Stelle einzufügen
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select  #erlaubt Pulldownmenus zu operieren
import time  # um Zeitpausen einzubauen, damit er z.Bsp. die Website laden kann
import pandas as pd
import re
from bs4 import BeautifulSoup

# öffnen der Website
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
driver.get('https://www.simap.ch')

time.sleep(3)

# ansteuern der Site mit den Zuschlägen (geht seltsamerweise nur mit "parttial_link_text")
zuschlaege = driver.find_element_by_partial_link_text('Zuschl')
zuschlaege.click()

# Eingrenzen der Daten von ... bis (clear, weil da standardmäsig was drinsteht)
datum1 = driver.find_element_by_name("STAT_TM_1")
datum1.clear()
datum1.send_keys("28.10.2004")

datum2 = driver.find_element_by_name("STAT_TM_2")
datum2.clear()
datum2.send_keys("07.10.2011")

# Selektieren der richtigen Checkboxen (Bau, Gemeinden, Kanton ZH)
buttons = driver.find_elements_by_name("TYPE_CONTRACT_CD_OB")
buttons[2].click()

boxes = driver.find_elements_by_name("AUTH_CD_OB")
boxes[0].click()
boxes[7].click()

box2 = driver.find_element_by_name("OB_CANTON_ZH")
box2.click()

# Das ist der Button, der die Recherche abschickt
button2 = driver.find_element_by_class_name("btn") 
button2.click()

# Jetzt poppt die Liste auf – diese Seite lesen wir raus, um mit Regex die Zahl der Treffer für den Loop abzugreifen:
main_page = driver.page_source.encode('utf-8')
regex = r"(?<=1-10 von )\d*"
trefferzahl = re.findall(regex, str(main_page))
treffer = int(trefferzahl[0])
treffer

# Jetzt klicken wir den Treffer Nummer 1 an (geht mit der "by_xpath"-Formel für a-tags)
firstentry = driver.find_element_by_xpath("//a[@href='/shabforms/servlet/Search?EID=2&ID=0']")
firstentry.click()

# Das ist der Loop: Seite als File abspeichern, dann auf den Button zum nächsten Ergebnis. 
for entries in range(0, (treffer-1)):
    page = driver.page_source.encode('utf-8')  
    with open("simap/zuschlag"+str(entries)+".htm", "wb+") as file:
        file.write(page)
        file.close()

    forward = driver.find_element_by_partial_link_text("Nächste Meldung")
    forward.click()